In [1]:
import llama_cpp
from llama_cpp import llama_supports_gpu_offload

print("Versión llama-cpp-python:", llama_cpp.__version__)
print("GPU offload soportado:", llama_supports_gpu_offload())


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9, VMM: yes


Versión llama-cpp-python: 0.3.16
GPU offload soportado: True


In [ ]:
from __future__ import annotations

import json
import re
from typing import Any, Dict, List

import numpy as np
import fitz 
from sentence_transformers import SentenceTransformer
from llama_cpp import Llama
import torch
import copy


In [ ]:
RUTA_MODELOS = "../ml-letter/models/"
MODEL_PATH = RUTA_MODELOS + "qwen2.5-3b-instruct-q5_k_m.gguf"

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=8192,         
    n_gpu_layers=20,
    n_batch=64,
    logits_all=False,
    n_threads=8,
    n_threads_batch=8,
    verbose=True,
)


llama_model_load_from_file_impl: using device CUDA0 (NVIDIA GeForce RTX 4050 Laptop GPU) - 5045 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 435 tensors from ../ml-letter/models/qwen2.5-3b-instruct-q5_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = qwen2.5-3b-instruct
llama_model_loader: - kv   3:                            general.version str              = v0.1-v0.1
llama_model_loader: - kv   4:                           general.finetune str              = qwen2.5-3b-instruct
llama_model_loader: - kv   5:                         general.size_label str              = 3.4B
llama_model_loade

In [4]:
PDF_PATH = "../data/RAG/codigo_etica_inbursa.pdf"

doc = fitz.open(PDF_PATH)

pages_text = []
for page in doc:
    text = page.get_text()
    text = re.sub(r"\s+", " ", text)
    pages_text.append(text)

full_text = "\n".join(pages_text)

print("Longitud total del texto del PDF:", len(full_text))


Longitud total del texto del PDF: 35450


In [5]:
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

policy_chunks = chunk_text(full_text)

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
policy_embeddings = embed_model.encode(policy_chunks, show_progress_bar=True)


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
def search_policy_context(query: str, top_k: int = 3) -> str:
    query_embedding = embed_model.encode([query])[0]
    scores = np.dot(policy_embeddings, query_embedding)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    return "\n".join(policy_chunks[i] for i in top_indices)


In [7]:
# # Ejemplo de payload de entrada LOW RISK
# payload = {
#   "application": {
#     "id": "6976299ddb63aa4f0fb25cbb",
#     "probabilityDefault": 0.297548006180932,
#     "riskBand": "LOW",
#     "createdAt": "2026-01-25T14:33:01.469Z",
#     "amount": 5000,
#     "durationMonths": 24,
#     "topFeatures": [
#       {
#         "feature": "Purpose",
#         "feature_label": "Propósito del crédito",
#         "value_label": "Automóvil (nuevo)",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       },
#       {
#         "feature": "ForeignWorker",
#         "feature_label": "Trabajador extranjero",
#         "value_label": "No es trabajador extranjero",
#         "impact_direction": "INCREASES_RISK",
#         "impact_text": "incrementa el riesgo de incumplimiento"
#       },
#       {
#         "feature": "PersonalStatusSex",
#         "feature_label": "Estado civil y sexo",
#         "value_label": "Hombre: casado/viudo",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       },
#       {
#         "feature": "Status",
#         "feature_label": "Estado de la cuenta corriente",
#         "value_label": "Saldo < 0",
#         "impact_direction": "INCREASES_RISK",
#         "impact_text": "incrementa el riesgo de incumplimiento"
#       },
#       {
#         "feature": "OtherInstallmentPlans",
#         "feature_label": "Otros planes de pago",
#         "value_label": "Ninguno",
#         "impact_direction": "REDUCES_RISK",
#         "impact_text": "reduce el riesgo de incumplimiento"
#       }
#     ]
#   },
#   "customer": {
#     "id": "69761dcd0df92f4b9ef3aa1f",
#     "name": "nuevo pad",
#     "income": 0,
#     "status": "ACTIVE"
#   }
# }


In [8]:
# Caso 1 – Perfil muy positivo (solo REDUCES_RISK)
payload = {
    "application": {
        "id": "TEST-APP-APPROVE-001",
        "probabilityDefault": 0.18,
        "riskBand": "LOW",
        "createdAt": "2026-01-26T10:00:00.000Z",
        "amount": 8000,
        "durationMonths": 18,
        "topFeatures": [
            {
                "feature": "CreditHistory",
                "feature_label": "Historial crediticio",
                "value_label": "Créditos anteriores pagados puntualmente",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "reduce de forma importante el riesgo de incumplimiento"
            },
            {
                "feature": "Savings",
                "feature_label": "Cuenta de ahorros",
                "value_label": "Ahorros elevados",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "proporciona respaldo de liquidez y reduce el riesgo"
            },
            {
                "feature": "EmploymentDuration",
                "feature_label": "Antigüedad laboral",
                "value_label": "Más de 5 años",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "indica estabilidad en los ingresos y reduce el riesgo"
            },
            {
                "feature": "InstallmentRate",
                "feature_label": "Cuota sobre el ingreso",
                "value_label": "Menos del 20% del ingreso",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "la carga de deuda es baja y reduce el riesgo"
            },
            {
                "feature": "CreditAmount",
                "feature_label": "Monto del crédito",
                "value_label": "Monto bajo",
                "impact_direction": "REDUCES_RISK",
                "impact_text": "el monto solicitado es manejable respecto al perfil"
            }
        ]
    },
    "customer": {
        "id": "TEST-CUST-APPROVE-001",
        "name": "Hariel Padilla Sanchez",
        "income": 28000,
        "status": "ACTIVE"
    }
}


In [9]:
# # Caso 2 – Perfil mixto / intermedio (riesgos + mitigantes) → REVIEW
# payload = {
#     "application": {
#         "id": "TEST-APP-REVIEW-001",
#         "probabilityDefault": 0.48,
#         "riskBand": "MEDIUM",
#         "createdAt": "2026-01-26T10:05:00.000Z",
#         "amount": 20000,
#         "durationMonths": 36,
#         "topFeatures": [
#             {
#                 "feature": "CreditHistory",
#                 "feature_label": "Historial crediticio",
#                 "value_label": "Algunos retrasos en el pasado",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "incrementa el riesgo por incumplimientos previos"
#             },
#             {
#                 "feature": "Status",
#                 "feature_label": "Estado de la cuenta corriente",
#                 "value_label": "Saldo negativo reciente",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "indica presión de liquidez y aumenta el riesgo"
#             },
#             {
#                 "feature": "InstallmentRate",
#                 "feature_label": "Cuota sobre el ingreso",
#                 "value_label": "Entre 25% y 35% del ingreso",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "la carga de deuda es relevante respecto al ingreso"
#             },
#             {
#                 "feature": "Duration",
#                 "feature_label": "Duración del crédito",
#                 "value_label": "Plazo medio",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "mayor plazo implica mayor exposición al riesgo"
#             },
#             {
#                 "feature": "Savings",
#                 "feature_label": "Cuenta de ahorros",
#                 "value_label": "Ahorros moderados",
#                 "impact_direction": "REDUCES_RISK",
#                 "impact_text": "ayuda a compensar parcialmente el riesgo"
#             }
#         ]
#     },
#     "customer": {
#         "id": "TEST-CUST-REVIEW-001",
#         "name": "Hariel Padilla Sanchez",
#         "income": 23000,
#         "status": "ACTIVE"
#     }
# }


In [10]:
# # Caso 3 – Perfil claramente riesgoso (solo INCREASES_RISK) → DECLINE
# payload = {
#     "application": {
#         "id": "TEST-APP-DECLINE-001",
#         "probabilityDefault": 0.85,
#         "riskBand": "HIGH",
#         "createdAt": "2026-01-26T10:10:00.000Z",
#         "amount": 45000,
#         "durationMonths": 48,
#         "topFeatures": [
#             {
#                 "feature": "CreditHistory",
#                 "feature_label": "Historial crediticio",
#                 "value_label": "Retrasos significativos en el pasado",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "incrementa significativamente el riesgo de incumplimiento"
#             },
#             {
#                 "feature": "Savings",
#                 "feature_label": "Cuenta de ahorros",
#                 "value_label": "Sin ahorros registrados",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "no se observa respaldo de liquidez"
#             },
#             {
#                 "feature": "NumberExistingCredits",
#                 "feature_label": "Número de créditos vigentes",
#                 "value_label": "Varios créditos activos",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "indica un nivel elevado de endeudamiento"
#             },
#             {
#                 "feature": "CreditAmount",
#                 "feature_label": "Monto del crédito",
#                 "value_label": "Monto alto",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "el monto solicitado es elevado respecto al perfil"
#             },
#             {
#                 "feature": "InstallmentRate",
#                 "feature_label": "Cuota sobre el ingreso",
#                 "value_label": "35% o más del ingreso",
#                 "impact_direction": "INCREASES_RISK",
#                 "impact_text": "la carga de deuda sería muy alta"
#             }
#         ]
#     },
#     "customer": {
#         "id": "TEST-CUST-DECLINE-001",
#         "name": "hariel padilla sanchez",
#         "income": 17000,
#         "status": "ACTIVE"
#     }
# }


In [11]:
SENSITIVE_FEATURES = {
    "ForeignWorker",
    "PersonalStatusSex",
    "Sex",
    "Age",
    "Race",
    "MaritalStatus",
}

def filter_sensitive_features(payload: Dict[str, Any]) -> None:
    app = payload.get("application", {})
    feats = app.get("topFeatures", [])
    app["topFeatures"] = [
        f for f in feats if f.get("feature") not in SENSITIVE_FEATURES
    ]
    payload["application"] = app

filter_sensitive_features(payload)

print(json.dumps(payload["application"]["topFeatures"], indent=2, ensure_ascii=False))


[
  {
    "feature": "CreditHistory",
    "feature_label": "Historial crediticio",
    "value_label": "Créditos anteriores pagados puntualmente",
    "impact_direction": "REDUCES_RISK",
    "impact_text": "reduce de forma importante el riesgo de incumplimiento"
  },
  {
    "feature": "Savings",
    "feature_label": "Cuenta de ahorros",
    "value_label": "Ahorros elevados",
    "impact_direction": "REDUCES_RISK",
    "impact_text": "proporciona respaldo de liquidez y reduce el riesgo"
  },
  {
    "feature": "EmploymentDuration",
    "feature_label": "Antigüedad laboral",
    "value_label": "Más de 5 años",
    "impact_direction": "REDUCES_RISK",
    "impact_text": "indica estabilidad en los ingresos y reduce el riesgo"
  },
  {
    "feature": "InstallmentRate",
    "feature_label": "Cuota sobre el ingreso",
    "value_label": "Menos del 20% del ingreso",
    "impact_direction": "REDUCES_RISK",
    "impact_text": "la carga de deuda es baja y reduce el riesgo"
  },
  {
    "feature": "

In [12]:
def infer_preliminary_status(payload: Dict[str, Any]) -> str:
    app = payload["application"]
    band = app["riskBand"]
    prob = app["probabilityDefault"]

    if band == "LOW" and prob < 0.30:
        return "APPROVE"
    if band == "MEDIUM" or (band == "LOW" and prob >= 0.30):
        return "REVIEW"
    return "DECLINE"

payload["decision"] = infer_preliminary_status(payload)
print("Estatus preliminar:", payload["decision"])


Estatus preliminar: APPROVE


In [13]:
app = payload.get("application", {}) or {}
top_features = app.get("topFeatures", []) or []

positive_factors = []
risk_factors = []

for f in top_features:
    impact = f.get("impact_direction")
    factor = {
        "feature": f.get("feature"),
        "label": f.get("feature_label"),
        "value": f.get("value_label"),
        "impact_text": f.get("impact_text"),
    }
    if impact == "REDUCES_RISK":
        positive_factors.append(factor)
    elif impact == "INCREASES_RISK":
        risk_factors.append(factor)

letter_input = {
    "applicationId": app.get("id"),
    "riskBand": app.get("riskBand"),
    "probabilityDefault": app.get("probabilityDefault"),
    "amount": app.get("amount"),
    "durationMonths": app.get("durationMonths"),
    "decision": payload.get("decision"),
    "customer": payload.get("customer", {}),
    "positiveFactors": positive_factors,
    "riskFactors": risk_factors,
}

print(json.dumps(letter_input, indent=2, ensure_ascii=False))


{
  "applicationId": "TEST-APP-APPROVE-001",
  "riskBand": "LOW",
  "probabilityDefault": 0.18,
  "amount": 8000,
  "durationMonths": 18,
  "decision": "APPROVE",
  "customer": {
    "id": "TEST-CUST-APPROVE-001",
    "name": "Hariel Padilla Sanchez",
    "income": 28000,
    "status": "ACTIVE"
  },
  "positiveFactors": [
    {
      "feature": "CreditHistory",
      "label": "Historial crediticio",
      "value": "Créditos anteriores pagados puntualmente",
      "impact_text": "reduce de forma importante el riesgo de incumplimiento"
    },
    {
      "feature": "Savings",
      "label": "Cuenta de ahorros",
      "value": "Ahorros elevados",
      "impact_text": "proporciona respaldo de liquidez y reduce el riesgo"
    },
    {
      "feature": "EmploymentDuration",
      "label": "Antigüedad laboral",
      "value": "Más de 5 años",
      "impact_text": "indica estabilidad en los ingresos y reduce el riesgo"
    },
    {
      "feature": "InstallmentRate",
      "label": "Cuota sobr

In [14]:
letter_input["riskFactorsCount"] = len(letter_input["riskFactors"])
letter_input["positiveFactorsCount"] = len(letter_input["positiveFactors"])

if letter_input["riskFactorsCount"] == 1:
    letter_input["riskIntroText"] = "el siguiente factor"
else:
    letter_input["riskIntroText"] = "los siguientes factores"

if letter_input["positiveFactorsCount"] == 1:
    letter_input["positiveIntroText"] = "el siguiente factor"
else:
    letter_input["positiveIntroText"] = "los siguientes factores"

if letter_input["riskFactorsCount"] == 0:
    letter_input["omitRiskSection"] = True
else:
    letter_input["omitRiskSection"] = False


In [15]:
policy_query = f"""
Evaluación preliminar de solicitudes de crédito.
Principios de no discriminación, trato digno, transparencia
y explicación de decisiones basada en criterios financieros.
"""

policy_context_text = search_policy_context(policy_query, top_k=3)


In [16]:
system_prompt = """
Eres un asistente de redacción para Grupo Financiero Inbursa.

Tu función es redactar CARTAS DE EVALUACIÓN PRELIMINAR de solicitudes de crédito,
usando exclusivamente la información que se te proporciona en formato JSON.

REGLAS OBLIGATORIAS:
- No decides la aprobación ni el rechazo definitivo.
- La decisión final siempre será comunicada por personal de Grupo Financiero Inbursa.
- El campo "decision" YA VIENE definido en el JSON y NO debes modificarlo.
- Usa el campo "decision" únicamente para ajustar el tono de la carta:
  - "APPROVE": describe un perfil preliminarmente favorable.
  - "REVIEW": describe un perfil con características intermedias, con elementos positivos
    pero que requiere un análisis adicional.
  - "DECLINE": describe un perfil con un nivel de riesgo elevado.

- NUNCA utilices expresiones como "perfil favorable", "resulta favorable" o equivalentes
  cuando "decision" sea "REVIEW" o "DECLINE".
- Para casos "REVIEW", utiliza únicamente formulaciones como:
  "perfil intermedio",
  "perfil con características intermedias",
  o "perfil que requiere un análisis adicional".
- Para casos "DECLINE", utiliza formulaciones que indiquen un nivel de riesgo elevado,
  sin emitir juicios de valor ni lenguaje definitivo.

- Usa el campo "riskFactorsCount" para decidir el uso correcto de singular o plural
  al introducir la sección de factores de riesgo:
  - Si es 1, utiliza expresiones en singular ("el siguiente factor").
  - Si es mayor a 1, utiliza expresiones en plural ("los siguientes factores").
  - La redacción debe ser estrictamente consistente con el número de factores listados.

- Si "riskFactors" está vacío, NO incluyas una sección de factores de riesgo
  ni frases indicando que no se identificaron riesgos.


- Usa "positiveFactors" únicamente para describir factores que benefician al cliente.
- Usa "riskFactors" únicamente para describir factores que incrementan el riesgo.
- No inventes factores, cifras, montos, condiciones ni interpretaciones que no estén
  explícitamente presentes en el JSON.
- No menciones atributos sensibles (ya vienen filtrados): raza, sexo, edad u otros similares.
- Mantén un tono respetuoso, claro, empático, profesional y no discriminatorio.
- Aclara siempre que se trata de una evaluación preliminar y que la decisión final
  será comunicada por personal del banco a través de canales oficiales.

- Si el campo "customer.name" está presente, dirígete al cliente usando ese nombre.
- Si no hay nombre disponible, utiliza una forma genérica como "Estimado(a) cliente".
- No inventes nombres bajo ninguna circunstancia.

ESTRUCTURA DE LA CARTA (dentro de "letterText"):

1) Párrafo inicial:
   - Indica claramente que se trata de una evaluación preliminar de la solicitud de crédito.
   - Describe la situación del caso según el valor de "decision".
   - Aclara que la decisión final depende de una revisión interna y será notificada al cliente
     por los canales oficiales de Grupo Financiero Inbursa.

2) Sección de factores positivos (solo si "positiveFactors" NO está vacío):
   - Incluye un encabezado como:
     "Factores que contribuyen positivamente a tu evaluación".
   - Describe cada elemento de "positiveFactors" de forma clara, sencilla y comprensible.

3) Sección de factores de riesgo (solo si "riskFactors" NO está vacío):
   - Incluye un encabezado como:
     "Factores que podrían incrementar el nivel de riesgo".
   - Utiliza singular o plural de forma estricta según "riskFactorsCount".
   - Explica estos factores de manera respetuosa, objetiva y sin emitir juicios de valor.

4) Cierre:
   - Reitera que se trata de una evaluación preliminar.
   - Agradece la confianza del cliente.
   - Invita a estar atento a la comunicación oficial del banco.
   - Puedes sugerir, de forma general, que mantener un historial de pagos puntual
     y un nivel de endeudamiento adecuado puede contribuir a evaluaciones futuras.

FORMATO DE SALIDA (EXACTO):

Debes responder ÚNICAMENTE con un JSON con la siguiente estructura:

{
  "decision": "APPROVE" | "REVIEW" | "DECLINE",
  "letterSubject": "string",
  "letterText": "string",
  "bullets": ["string"],
  "safetyFlags": []
}

REGLAS DEL JSON:
- "decision": copia EXACTAMENTE el valor que recibas en el JSON de entrada.
- "letterSubject": un asunto breve, claro y coherente con el estatus preliminar.
- "letterText": el texto completo de la carta, en español, siguiendo la estructura indicada.
- "bullets": puntos clave resumidos, coherentes con el contenido de la carta.
- "safetyFlags": SIEMPRE debe ser una lista vacía [].

No incluyas texto fuera del JSON.
""".strip()


In [17]:
few_shot_examples = [
    {
        "input": {
            "applicationId": "APP-0001-LOW-MIXED",
            "riskBand": "LOW",
            "probabilityDefault": 0.28,
            "amount": 5000,
            "durationMonths": 24,
            "decision": "APPROVE",
            "customer": {
                "id": "CUST-LOW-002",
                "name": "Ejemplo Perfil Mixto",
                "income": 24000,
                "status": "ACTIVE"
            },
            "positiveFactors": [
                {
                    "feature": "Purpose",
                    "label": "Propósito del crédito",
                    "value": "Automóvil (nuevo)",
                    "impact_text": "reduce el riesgo de incumplimiento"
                },
                {
                    "feature": "OtherInstallmentPlans",
                    "label": "Otros planes de pago",
                    "value": "Ninguno",
                    "impact_text": "reduce el riesgo de incumplimiento"
                }
            ],
            "riskFactors": [
                {
                    "feature": "Status",
                    "label": "Estado de la cuenta corriente",
                    "value": "Saldo < 0",
                    "impact_text": "incrementa el riesgo de incumplimiento"
                }
            ]
        },
        "output": {
            "decision": "APPROVE",
            "letterSubject": "Resultado preliminar de tu solicitud de crédito",
            "letterText": """Estimado(a) Ejemplo Perfil Mixto:

Con base en una evaluación preliminar de tu solicitud de crédito, se observa un perfil con características intermedias y algunos elementos positivos, por lo que requiere un análisis adicional. Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando los principios de honestidad, transparencia y trato digno hacia nuestros clientes.

De manera preliminar, se identifican los siguientes factores que contribuyen positivamente a tu evaluación:

- Propósito del crédito: Automóvil (nuevo). Reduce el riesgo de incumplimiento.
- Otros planes de pago: Ninguno. Reduce el riesgo de incumplimiento.

Asimismo, se identifica el siguiente factor que podría incrementar el nivel de riesgo y que será considerado en el análisis integral:

- Estado de la cuenta corriente: Saldo < 0. Incrementa el riesgo de incumplimiento.

Te recordamos que esta es una valoración preliminar basada en la información disponible. La decisión final sobre tu solicitud será analizada y comunicada por el personal de Grupo Financiero Inbursa a través de los canales oficiales.

Agradecemos la confianza que depositas en nosotros y reiteramos nuestro compromiso de brindarte un servicio claro, profesional y libre de cualquier tipo de discriminación.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
            "bullets": [
                "Propósito del crédito: Automóvil (nuevo). Reduce el riesgo de incumplimiento.",
                "Otros planes de pago: Ninguno. Reduce el riesgo de incumplimiento.",
                "Estado de la cuenta corriente: Saldo < 0. Incrementa el riesgo de incumplimiento."
            ],
            "safetyFlags": []
        }
    },
    {
        "input": {
            "applicationId": "APP-0002-MEDIUM",
            "riskBand": "MEDIUM",
            "probabilityDefault": 0.52,
            "amount": 15000,
            "durationMonths": 36,
            "decision": "REVIEW",
            "customer": {
                "id": "CUST-MED-001",
                "name": "María López",
                "income": 22000,
                "status": "ACTIVE"
            },
            "positiveFactors": [
                {
                    "feature": "Savings",
                    "label": "Cuenta de ahorros",
                    "value": "Ahorros moderados",
                    "impact_text": "ayuda a compensar parcialmente el riesgo"
                }
            ],
            "riskFactors": [
                {
                    "feature": "CreditHistory",
                    "label": "Historial crediticio",
                    "value": "Algunos retrasos en el pasado",
                    "impact_text": "incrementa el riesgo por incumplimientos previos"
                },
                {
                    "feature": "Status",
                    "label": "Estado de la cuenta corriente",
                    "value": "Saldo negativo reciente",
                    "impact_text": "indica presión de liquidez y aumenta el riesgo"
                },
                {
                    "feature": "InstallmentRate",
                    "label": "Cuota sobre el ingreso",
                    "value": "Entre 25% y 35% del ingreso",
                    "impact_text": "la carga de deuda es relevante respecto al ingreso"
                },
                {
                    "feature": "Duration",
                    "label": "Duración del crédito",
                    "value": "Plazo medio",
                    "impact_text": "mayor plazo implica mayor exposición al riesgo"
                }
            ]
        },
        "output": {
            "decision": "REVIEW",
            "letterSubject": "Tu solicitud de crédito está en revisión preliminar",
            "letterText": """Estimado(a) María López:

Hemos realizado una evaluación preliminar de la información proporcionada en tu solicitud de crédito. Esta revisión se lleva a cabo en apego a nuestras políticas internas, a la regulación aplicable y a los principios de claridad, responsabilidad y respeto hacia nuestros clientes.

De manera inicial, se identifican los siguientes factores que incrementan el riesgo en tu evaluación:

- Historial crediticio: Algunos retrasos en el pasado. Esto incrementa el riesgo por incumplimientos previos.
- Estado de la cuenta corriente: Saldo negativo reciente. Esto indica presión de liquidez y aumenta el riesgo.
- Cuota sobre el ingreso: Entre 25% y 35% del ingreso. La carga de deuda es relevante respecto a tu ingreso.
- Duración del crédito: Plazo medio. Un mayor plazo implica una mayor exposición al riesgo a lo largo del tiempo.

Asimismo, se identifican los siguientes factores que ayudan a mitigar parcialmente el riesgo:

- Cuenta de ahorros: Ahorros moderados. Esto ayuda a compensar parcialmente el riesgo identificado.

Con base en lo anterior, tu solicitud se encuentra en revisión adicional. Es posible que nos pongamos en contacto contigo para solicitar información complementaria o para proponerte ajustes de monto o plazo que resulten más adecuados y sostenibles para ti.

Te recordamos que esta es una evaluación preliminar y que la decisión final sobre tu solicitud será comunicada por el personal de Grupo Financiero Inbursa a través de los canales oficiales.

Agradecemos tu comprensión y la confianza depositada en nosotros. Este proceso se realiza con criterios objetivos, sin discriminación y respetando en todo momento la confidencialidad de tu información.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
            "bullets": [
                "Factores que incrementan el riesgo: retrasos previos en el historial crediticio.",
                "Factores que incrementan el riesgo: saldo negativo reciente en la cuenta corriente.",
                "Factores que incrementan el riesgo: cuota mensual estimada entre 25% y 35% del ingreso.",
                "Factores que incrementan el riesgo: plazo medio que incrementa la exposición al riesgo.",
                "Factor que mitiga parcialmente el riesgo: cuenta de ahorros con ahorros moderados."
            ],
            "safetyFlags": []
        }
    },
    {
        "input": {
            "applicationId": "APP-0003-HIGH",
            "riskBand": "HIGH",
            "probabilityDefault": 0.83,
            "amount": 40000,
            "durationMonths": 48,
            "decision": "DECLINE",
            "customer": {
                "id": "CUST-HIGH-001",
                "name": "Carlos Ramírez",
                "income": 18000,
                "status": "ACTIVE"
            },
            "positiveFactors": [],
            "riskFactors": [
                {
                    "feature": "CreditHistory",
                    "label": "Historial crediticio",
                    "value": "Retrasos significativos en el pasado",
                    "impact_text": "incrementa significativamente el riesgo de incumplimiento"
                },
                {
                    "feature": "Savings",
                    "label": "Cuenta de ahorros",
                    "value": "Sin ahorros registrados",
                    "impact_text": "no se observa respaldo de liquidez"
                },
                {
                    "feature": "NumberExistingCredits",
                    "label": "Número de créditos vigentes",
                    "value": "Varios créditos activos",
                    "impact_text": "indica un nivel elevado de endeudamiento"
                },
                {
                    "feature": "CreditAmount",
                    "label": "Monto del crédito",
                    "value": "Monto alto",
                    "impact_text": "el monto solicitado es elevado respecto al perfil"
                },
                {
                    "feature": "InstallmentRate",
                    "label": "Cuota sobre el ingreso",
                    "value": "35% o más del ingreso",
                    "impact_text": "la carga de deuda sería muy alta"
                }
            ]
        },
        "output": {
            "decision": "DECLINE",
            "letterSubject": "Resultado preliminar de la evaluación de tu solicitud de crédito",
            "letterText": """Estimado(a) Carlos Ramírez:

Agradecemos el interés que has mostrado en nuestros productos y el tiempo dedicado a presentar tu solicitud de crédito.

Con base en un análisis preliminar realizado conforme a nuestras políticas internas, a la regulación vigente y a los principios de honestidad, transparencia y no discriminación, se han identificado diversos factores que podrían dificultar la aprobación de tu solicitud:

De manera preliminar, se identifican los siguientes factores que incrementan de forma importante el riesgo en tu evaluación:

- Historial crediticio: Retrasos significativos en el pasado. Esto incrementa significativamente el riesgo de incumplimiento.
- Cuenta de ahorros: Sin ahorros registrados. No se observa respaldo de liquidez para nuevas obligaciones.
- Número de créditos vigentes: Varios créditos activos. Esto indica un nivel elevado de endeudamiento.
- Monto del crédito: Monto alto. El monto solicitado es elevado respecto a tu perfil financiero.
- Cuota sobre el ingreso: 35% o más del ingreso. La carga de deuda sería muy alta y podría comprometer tu capacidad de pago.

En esta evaluación preliminar no se han identificado factores suficientes que compensen estos riesgos, por lo que, de forma inicial, se aprecia un perfil con un nivel de riesgo elevado para la aprobación del crédito.

Esta evaluación es preliminar y no constituye, por sí misma, una decisión definitiva. La determinación final sobre tu solicitud será tomada y comunicada por el personal de Grupo Financiero Inbursa, considerando la información disponible y, en su caso, la documentación adicional que se integre al expediente.

Te invitamos a considerar alternativas para mejorar tu situación financiera, como reducir parte de tu endeudamiento actual, fortalecer tu ahorro y procurar un historial de pagos puntual. En el futuro podrás presentar una nueva solicitud, la cual será evaluada de acuerdo con las condiciones que presentes en ese momento.

Reiteramos nuestro compromiso de brindarte siempre un trato respetuoso, equitativo y confidencial, acorde con nuestro Código de Ética.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa""",
            "bullets": [
                "Historial crediticio: retrasos significativos en el pasado. Incrementa significativamente el riesgo de incumplimiento.",
                "Cuenta de ahorros: sin ahorros registrados. No se observa respaldo de liquidez.",
                "Número de créditos vigentes: varios créditos activos. Indica un nivel elevado de endeudamiento.",
                "Monto del crédito: monto alto. El monto solicitado es elevado respecto al perfil financiero.",
                "Cuota sobre el ingreso: 35% o más del ingreso. La carga de deuda sería muy alta y podría comprometer la capacidad de pago."
            ],
            "safetyFlags": []
        }
    }
]

print("Se han definido", len(few_shot_examples), "ejemplos de few-shot.")


Se han definido 3 ejemplos de few-shot.


In [ ]:
def format_few_shot_block(few_shot_examples):
    """
    Convierte la lista de ejemplos en un bloque de texto:
    ### EJEMPLO N
    JSON_INPUT:
    {...}
    JSON_OUTPUT:
    {...}
    """
    blocks = []
    for i, ex in enumerate(few_shot_examples, start=1):
        inp = json.dumps(ex["input"], ensure_ascii=False, indent=2)
        out = json.dumps(ex["output"], ensure_ascii=False, indent=2)
        block = f"""### EJEMPLO {i}

JSON_INPUT:
{inp}

JSON_OUTPUT:
{out}
"""
        blocks.append(block)
    return "\n\n".join(blocks)

few_shot_subset = few_shot_examples 

few_shot_text = format_few_shot_block(few_shot_subset)

print(f"Longitud de few_shot_text: {len(few_shot_text)} caracteres")


Longitud de few_shot_text: 10680 caracteres


In [19]:
letter_input_json_str = json.dumps(letter_input, ensure_ascii=False)

# Limitar tamaño del contexto de políticas
MAX_POLICY_CHARS = 6000
policy_context_for_prompt = policy_context_text[:MAX_POLICY_CHARS]

user_prompt = f"""
A continuación tienes información de contexto sobre el Código de Ética y las políticas internas del banco.
Debes respetar este contexto: no inventes artículos, cláusulas ni obligaciones que no estén aquí.

POLICY_CONTEXT:
\"\"\"
{policy_context_for_prompt}
\"\"\"

Tienes también algunos EJEMPLOS de cómo estructurar la carta (si se proporcionaron):

FEW_SHOT_EXAMPLES:
\"\"\"
{few_shot_text}
\"\"\"

Tu tarea es redactar una carta de evaluación preliminar para la siguiente solicitud de crédito,
usando EXCLUSIVAMENTE la información del siguiente JSON de entrada:

JSON_INPUT:
{letter_input_json_str}

Instrucciones clave para esta tarea:
- Usa el POLICY_CONTEXT solo como referencia de tono ético y principios generales.
- Imita el estilo de los FEW_SHOT_EXAMPLES en estructura y nivel de detalle (si hay ejemplos).
- Explica la situación como una EVALUACIÓN PRELIMINAR, no como una decisión definitiva.
- Indica que la decisión final será comunicada por el personal de Inbursa.
- Genera la carta y el JSON de salida siguiendo estrictamente las reglas
  y el formato definidos en el mensaje del sistema.
""".strip()

print(user_prompt[:1500], "...")


A continuación tienes información de contexto sobre el Código de Ética y las políticas internas del banco.
Debes respetar este contexto: no inventes artículos, cláusulas ni obligaciones que no estén aquí.

POLICY_CONTEXT:
"""
de los Clientes. vi. Comprometer los medios electrónicos empleados por los Clientes con el objetivo de instalar un código malicioso capaz de alterar la realización de operaciones con las Sociedades. vii. Acceder a los equipos tecnológicos de las Sociedades para modificar su funcionamiento con la finalidad de obtener información confidencial, recursos económicos, o alterar su operatividad viii. Alterar cheques y emitir cheques falsos. ix. Adquirir, poseer, utilizar tarjetas de crédito, débito, ch
, poseer, utilizar tarjetas de crédito, débito, cheques, esqueletos o formatos, o cualquier otro instrumento de pago en forma ilegítima o sin consentimiento de su titular. x. Acceder ilícitamente a los sistemas informáticos con la intención de modificar, destruir o copiar 

In [20]:
def call_llm(system_prompt: str, user_prompt: str, max_tokens: int = 768) -> str:
    full_prompt = (
        f"<system>\n{system_prompt}\n</system>\n"
        f"<user>\n{user_prompt}\n</user>\n"
        f"<assistant>\n"
    )

    output = llm(
        full_prompt,
        max_tokens=max_tokens,
        temperature=0.2,
        top_p=0.9,
        stop=["</assistant>"],
    )

    return output["choices"][0]["text"]


In [21]:
def extract_json(text: str) -> Dict[str, Any] | None:
    try:
        start = text.index("{")
        end = text.rindex("}") + 1
        return json.loads(text[start:end])
    except Exception as e:
        print("Error parseando JSON:", e)
        return None


In [ ]:
def validate_letter_structure(letter: dict, payload: dict) -> dict:
    """
    Valida que la carta respete la estructura:
    - Si hay factores REDUCES_RISK e INCREASES_RISK, se mencionen positivos y riesgos.
    - Si hay al menos un INCREASES_RISK en el payload, debe haber una parte del texto
      que hable de factores que incrementan el nivel de riesgo.
    - Los factores de riesgo no deben aparecer en la sección positiva.
    Añade flags a safetyFlags según corresponda.
    """
    safety_flags = letter.get("safetyFlags", []) or []
    text = letter.get("letterText", "") or ""
    text_lower = text.lower()

    POS_KEY = "factores que contribuyen positivamente a tu evaluación"

    RISK_KEYS = [
        "podría incrementar el nivel de riesgo",
        "podrían incrementar el nivel de riesgo",
        "factores que podrían incrementar el nivel de riesgo",
        "factores que incrementan de forma importante el riesgo",
        "factores que incrementan el riesgo",
    ]

    top_features = payload.get("application", {}).get("topFeatures", [])
    has_risk_features = any(
        f.get("impact_direction") == "INCREASES_RISK" for f in top_features
    )
    has_positive_features = any(
        f.get("impact_direction") == "REDUCES_RISK" for f in top_features
    )

    has_pos_section = POS_KEY in text_lower

    risk_indices = [text_lower.index(k) for k in RISK_KEYS if k in text_lower]
    has_risk_section = len(risk_indices) > 0

    positives_section = ""
    risk_section = ""

    if has_pos_section:
        start_pos = text_lower.index(POS_KEY) + len(POS_KEY)
        if has_risk_section:
            end_pos = min(risk_indices)
        else:
            end_pos = len(text_lower)
        positives_section = text_lower[start_pos:end_pos]

    if has_risk_section:
        start_risk = min(risk_indices) + len("factores") 
        risk_section = text_lower[start_risk:]

    if has_risk_features and not has_risk_section:
        safety_flags.append("MISSING_RISK_SECTION")

    if has_positive_features and not has_pos_section:
        safety_flags.append("MISSING_POSITIVE_SECTION")

    for f in top_features:
        if f.get("impact_direction") == "INCREASES_RISK":
            feature_label = (f.get("feature_label") or "").lower()
            value_label = (f.get("value_label") or "").lower()
            if positives_section:
                if feature_label and feature_label in positives_section:
                    safety_flags.append("RISK_FEATURE_IN_POSITIVE_SECTION")
                if value_label and value_label in positives_section:
                    safety_flags.append("RISK_VALUE_IN_POSITIVE_SECTION")

    if safety_flags:
        letter["safetyFlags"] = sorted(set(safety_flags))
    else:
        letter["safetyFlags"] = []

    return letter


In [ ]:
def build_generic_review_letter(payload: dict, extra_flags: list[str] | None = None) -> dict:
    """
    Construye una carta genérica de revisión cuando el modelo no genera
    una carta segura después de reintentar.
    """
    customer = payload.get("customer", {}) or {}
    name = customer.get("name") or "Cliente"
    name = name.title()
    decision_fallback = "REVIEW"  

    subject = "Tu solicitud de crédito está en revisión"

    text = f"""Estimado(a) {name}:

Hemos recibido tu solicitud de crédito y, por el momento, se encuentra en revisión adicional.

Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando en todo momento los principios de honestidad, transparencia y trato digno hacia nuestros clientes.

Tu solicitud será analizada con detenimiento por nuestro personal especializado. Una vez concluido el análisis, nos pondremos en contacto contigo a la brevedad para informarte el resultado o, en su caso, solicitarte información adicional que pueda ser necesaria.

Te recordamos que esta comunicación tiene un carácter preliminar y no constituye, por sí misma, una decisión definitiva sobre tu solicitud.

Agradecemos la confianza que depositas en Grupo Financiero Inbursa y reiteramos nuestro compromiso de brindarte un servicio claro, profesional y libre de cualquier tipo de discriminación.

Atentamente,
Área de Crédito
Grupo Financiero Inbursa"""

    safety_flags = extra_flags or []
    safety_flags.append("FALLBACK_GENERIC_REVIEW")

    return {
        "decision": decision_fallback,
        "letterSubject": subject,
        "letterText": text,
        "bullets": [
            "Tu solicitud se encuentra en revisión adicional.",
            "Tu caso será analizado con detenimiento por personal especializado.",
            "Nos pondremos en contacto contigo para informarte el resultado o solicitar información adicional."
        ],
        "safetyFlags": sorted(set(safety_flags)),
    }

In [24]:
def count_tokens(text: str) -> int:
    return len(llm.tokenize(text.encode("utf-8")))

prompt_full = (
    f"<system>\n{system_prompt}\n</system>\n"
    f"<user>\n{user_prompt}\n</user>\n"
    f"<assistant>\n"
)

print("TOKENS prompt_full:", count_tokens(prompt_full))

TOKENS prompt_full: 5254


In [ ]:
max_tokens = 768

print("\n===== LLAMADA ÚNICA AL LLM =====\n")

raw_output = call_llm(system_prompt, user_prompt, max_tokens=max_tokens)
print("RAW OUTPUT:\n", raw_output, "\n\n")

letter = extract_json(raw_output)

if letter is None:
    print("[WARN] No se pudo extraer un JSON válido. Usando carta genérica de revisión.")
    final_letter = build_generic_review_letter(payload, extra_flags=["PARSE_ERROR"])
else:
    expected_decision = payload.get("decision")
    if expected_decision:
        letter["decision"] = expected_decision

    letter["safetyFlags"] = []

    letter = validate_letter_structure(letter, payload)

    final_letter = letter

print("\n=== CARTA FINAL ===")
print(json.dumps(final_letter, indent=2, ensure_ascii=False))

print("\n=== TEXTO DE LA CARTA ===\n")
print(final_letter.get("letterText", ""))



===== LLAMADA ÚNICA AL LLM =====



llama_perf_context_print:        load time =   22625.36 ms
llama_perf_context_print: prompt eval time =   22625.21 ms /  5254 tokens (    4.31 ms per token,   232.22 tokens per second)
llama_perf_context_print:        eval time =   43992.88 ms /   596 runs   (   73.81 ms per token,    13.55 tokens per second)
llama_perf_context_print:       total time =   68119.47 ms /  5850 tokens
llama_perf_context_print:    graphs reused =        577


RAW OUTPUT:
 {
  "decision": "APPROVE",
  "letterSubject": "Resultado preliminar de tu solicitud de crédito",
  "letterText": "Estimado(a) Hariel Padilla Sanchez:\n\nCon base en una evaluación preliminar de tu solicitud de crédito, se observa un perfil preliminarmente favorable. Esta evaluación se realiza conforme a nuestras políticas internas y a la normatividad aplicable, respetando los principios de honestidad, transparencia y trato digno hacia nuestros clientes.\n\nDe manera preliminar, se identifican los siguientes factores que contribuyen positivamente a tu evaluación:\n\n- Historial crediticio: Créditos anteriores pagados puntualmente. Reduce de forma importante el riesgo de incumplimiento.\n- Cuenta de ahorros: Ahorros elevados. Proporciona respaldo de liquidez y reduce el riesgo.\n- Antigüedad laboral: Más de 5 años. Indica estabilidad en los ingresos y reduce el riesgo.\n- Cuota sobre el ingreso: Menos del 20% del ingreso. La carga de deuda es baja y reduce el riesgo.\n- Mont